In [2]:
import os
import cx_Oracle
import pandas as pd
import numpy as np
from datetime import date,datetime,time
#import xgboost as xgb
import warnings
import pickle
import time
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score 
from inspect import signature
from sklearn.preprocessing import PolynomialFeatures
import math

In [136]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\PayU CLX Scoring")
df=pd.read_pickle(os.getcwd()+"\\data files\\payu_tenure_predictors_fstpd30_skp_case30_Sep_12hrs.pkl")


In [152]:
#pickle.dump(model, open(os.getcwd()+"\\Data Files\\model_name.sav", 'wb'))
loaded_model = pickle.load(open("xgb_fst_clx_6pred_derived_final_model.sav", 'rb'))


In [138]:
X=df[['AFFLUENCE_SCORE','ENGAGEMENT_SCORE','DIGITAL_RISK_SCORE','ACTIVE_MONTHS_365',
      'TOTAL_AMT_SALE_365','TOTAL_CNT_SALE_365','RISK_SCORE','SKP_CREDIT_CASE']]
Y=df['RISK_FSTPD30']


In [139]:
X['ACTIVE_MONTHS_365_BUC'] = X['ACTIVE_MONTHS_365'].apply(lambda symbol: 
 '0' if math.isnan(symbol) or symbol == 0
 else '1-3' if symbol> 0 and symbol<=3
 else '4-6' if symbol> 3 and symbol<=6
 else '7-9' if symbol> 6 and symbol<=9 
 else '>9')

X['TOTAL_AMT_SALE_365_BUC'] = X['TOTAL_AMT_SALE_365'].apply(lambda symbol: 
 '0' if math.isnan(symbol) or symbol == 0
 else '1-1000' if symbol> 0 and symbol<=1000
 else '1001-5000' if symbol> 1000 and symbol<=5000
 else '5001-10000' if symbol> 5000 and symbol<=10000 
 else '>10000')

X['TOTAL_CNT_SALE_365_BUC'] = X['TOTAL_CNT_SALE_365'].apply(lambda symbol: 
 '0' if math.isnan(symbol) or symbol == 0
 else '1-3' if symbol> 0 and symbol<=3
 else '4-6' if symbol> 3 and symbol<=6
 else '7-10' if symbol> 6 and symbol<=10 
 else '>10')


X['ACTIVE_MONTHS_365_ORD'] = X['ACTIVE_MONTHS_365'].apply(lambda symbol: 
 0 if math.isnan(symbol) or symbol == 0
 else 2 if symbol> 0 and symbol<=3
 else 5 if symbol> 3 and symbol<=6
 else 8 if symbol> 6 and symbol<=9 
 else 11)

X['TOTAL_AMT_SALE_365_ORD'] = X['TOTAL_AMT_SALE_365'].apply(lambda symbol: 
 0 if math.isnan(symbol) or symbol == 0
 else 500 if symbol> 0 and symbol<=1000
 else 3000 if symbol> 1000 and symbol<=5000
 else 7500 if symbol> 5000 and symbol<=10000 
 else 35000)

X['TOTAL_CNT_SALE_365_ORD'] = X['TOTAL_CNT_SALE_365'].apply(lambda symbol: 
 0 if math.isnan(symbol) or symbol == 0
 else 2 if symbol> 0 and symbol<=3
 else 5 if symbol> 3 and symbol<=6
 else 8.5 if symbol> 6 and symbol<=10 
 else 20)

    
X["AF_div_EN"]=X["AFFLUENCE_SCORE"]/X["ENGAGEMENT_SCORE"]
X["AF_div_DR"]=X["AFFLUENCE_SCORE"]/X["DIGITAL_RISK_SCORE"]
X["EN_div_AF"]=X["ENGAGEMENT_SCORE"]/X["AFFLUENCE_SCORE"]

X["EN_div_DR"]=X["ENGAGEMENT_SCORE"]/X["DIGITAL_RISK_SCORE"]
X["DR_div_AF"]=X["DIGITAL_RISK_SCORE"]/X["AFFLUENCE_SCORE"]
X["DR_div_EN"]=X["DIGITAL_RISK_SCORE"]/X["ENGAGEMENT_SCORE"]

X["AF_mul_EN"]=X["AFFLUENCE_SCORE"]*X["ENGAGEMENT_SCORE"]
X["EN_mul_DR"]=X["ENGAGEMENT_SCORE"]*X["DIGITAL_RISK_SCORE"]
X["DR_mul_AF"]=X["DIGITAL_RISK_SCORE"]*X["AFFLUENCE_SCORE"]
    
X['AVG_TICKET'] = X["TOTAL_AMT_SALE_365_ORD"]/X["TOTAL_CNT_SALE_365_ORD"]
X['MONTHLY_AMT'] = X["TOTAL_AMT_SALE_365_ORD"]/X["ACTIVE_MONTHS_365_ORD"]
X['MONTHLY_CNT'] = X["TOTAL_CNT_SALE_365_ORD"]/X["ACTIVE_MONTHS_365_ORD"]



C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [140]:
missing_value=None; cpu=-1; test_size=0.3; early_stopping_round=40; split_random_state=None;

In [141]:
X_final=X[['AFFLUENCE_SCORE','ENGAGEMENT_SCORE','DIGITAL_RISK_SCORE','ACTIVE_MONTHS_365_ORD',
      'TOTAL_AMT_SALE_365_ORD','TOTAL_CNT_SALE_365_ORD','RISK_SCORE','AVG_TICKET',
       'AF_mul_EN', 'EN_mul_DR', 'DR_mul_AF', 'MONTHLY_AMT', 'MONTHLY_CNT',
       'AF_div_EN', 'AF_div_DR', 'EN_div_AF', 'EN_div_DR', 'DR_div_AF',
       'DR_div_EN','SKP_CREDIT_CASE']]


In [142]:
y_pred  = loaded_model.predict_proba(X_final.drop(columns = ['SKP_CREDIT_CASE','RISK_SCORE']))[:,1]

In [143]:
X_final['PAYU_PRED']  = y_pred

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [147]:
metric_auc = metrics.roc_auc_score(Y, X_final['PAYU_PRED'])
gini = 2 * metric_auc - 1
print(gini)

0.08537233138972211


In [148]:
#Fit Linear Model to calculate gini lift
X_final.columns
X_final

df_payu_score = X_final
df_payu_score['RISK_FSTPD30'] = Y
df_payu_score.dropna(subset=['RISK_SCORE'],inplace = True)

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [149]:
lm = linear_model.LinearRegression().fit(df_payu_score[['RISK_SCORE','PAYU_PRED']],df_payu_score["RISK_FSTPD30"])

ypred= lm.predict(df_payu_score[['RISK_SCORE','PAYU_PRED']])

metric_auc = metrics.roc_auc_score(df_payu_score["RISK_FSTPD30"], ypred)
gini = 2 * metric_auc - 1
print(gini)

0.34351275639485235


In [150]:
metric_auc = metrics.roc_auc_score(df_payu_score["RISK_FSTPD30"], df_payu_score["RISK_SCORE"])
gini = 2 * metric_auc - 1
print(gini)

0.34040252213799693


In [151]:
poly = PolynomialFeatures(1)
new_df = pd.DataFrame(poly.fit_transform(df_payu_score[['RISK_SCORE','PAYU_PRED']]))
new_df.columns = poly.get_feature_names(input_features = ['RISK_SCORE','PAYU_PRED'])

#lm = linear_model.LinearRegression().fit(new_df,df_payu_score["RISK_FSTPD30"])
lm = linear_model.LogisticRegression().fit(new_df,df_payu_score["RISK_FSTPD30"])

ypred= lm.predict(new_df)

metric_auc = metrics.roc_auc_score(df_payu_score["RISK_FSTPD30"], ypred)
gini = 2 * metric_auc - 1
print(gini)

print(lm.coef_)

print(lm.intercept_)


0.0
[[-1.80980239  5.56679202 -0.38189413]]
[-1.80980239]
